# 🎧 Spotify MP3 Converter
Converte músicas, álbuns e playlists do **Spotify** em **MP3**, usando o **YouTube** como fonte.

Este notebook é totalmente automático — basta fornecer um link do Spotify e fazer upload dos cookies do YouTube.

## 👤 Autor
**kiraenes**

## ⚠️ Aviso Legal
Este projeto é apenas para fins **educacionais**.
Respeite os direitos autorais das plataformas envolvidas.

## 🚀 Como usar
1. Faça upload do arquivo `keys.env` com:
```
SPOTIFY_CLIENT_ID=xxxx
SPOTIFY_CLIENT_SECRET=xxxx
```
2. Faça upload do `cookies.txt` exportado do YouTube.
3. Cole o link do Spotify.
4. Aguarde o ZIP final.

---
### 📦 Tudo o código está abaixo (numa só célula)

In [ ]:
!pip install -q spotipy mutagen yt-dlp python-dotenv

In [ ]:
import os, shutil, queue, threading
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, TIT2, TPE1, TALB, APIC
from google.colab import files
from time import sleep
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import yt_dlp

print("📁 Upload your keys.env with Spotify API keys:")
uploaded = files.upload()
env_path = list(uploaded.keys())[0]
load_dotenv(env_path)

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
if not SPOTIFY_CLIENT_ID or not SPOTIFY_CLIENT_SECRET:
    raise ValueError("❌ SPOTIFY_CLIENT_ID or SPOTIFY_CLIENT_SECRET missing!")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
))

print("📁 Upload your YouTube cookies.txt (from ChocChipCookie extension):")
uploaded_cookies = files.upload()
COOKIE_PATH = list(uploaded_cookies.keys())[0]

def tag_mp3(path, title, artist, album, img_url=None):
    try:
        audio = MP3(path, ID3=ID3)
        audio.add_tags()
    except:
        pass
    audio.tags.add(TIT2(encoding=3, text=title))
    audio.tags.add(TPE1(encoding=3, text=artist))
    audio.tags.add(TALB(encoding=3, text=album))
    if img_url:
        try:
            import requests
            img = requests.get(img_url, timeout=5).content
            audio.tags.add(APIC(encoding=3, mime='image/jpeg', type=3, desc='Cover', data=img))
        except:
            pass
    audio.save()

counter = 0

def download_worker(q, folder):
    global counter
    while True:
        try:
            track = q.get(timeout=2)
        except queue.Empty:
            break

        counter += 1
        num = f"{counter}º"

        title = track['name']
        artist = track['artists'][0]['name']
        album = track['album']['name']
        cover = track['album']['images'][0]['url'] if track['album']['images'] else None

        search_query = f"ytsearch1:{title} {artist}"
        output_path = os.path.join(folder, f"{title} - {artist}.%(ext)s")

        class YTDLPLogger:
            def debug(self, msg): pass
            def warning(self, msg): pass
            def error(self, msg): pass

        def my_hook(d):
            if d['status'] == 'finished':
                print(f"🎵 {num} {title} - {artist} ✅")
            elif d['status'] == 'error':
                print(f"🎵 {num} {title} - {artist} ❌")

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': output_path,
            'cookiefile': COOKIE_PATH,
            'noplaylist': True,
            'logger': YTDLPLogger(),
            'progress_hooks': [my_hook],
            'quiet': True,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }]
        }

        try:
            with yt_dlp.YoutubeDL({**ydl_opts, 'extract_flat': True}) as ydl:
                info_dict = ydl.extract_info(search_query, download=False)

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([search_query])

            mp3_path = os.path.join(folder, f"{title} - {artist}.mp3")
            tag_mp3(mp3_path, title, artist, album, cover)

        except Exception:
            print(f"🎵 {num} {title} - {artist} ❌")
        finally:
            q.task_done()

def download_spotify_from_youtube(url, max_threads=5):
    if "playlist" in url:
        info = sp.playlist(url)
        name = info['name']
        results = sp.playlist_items(url)
        tracks = []
        while results:
            tracks.extend([i['track'] for i in results['items'] if i['track']])
            results = sp.next(results)
    elif "album" in url:
        info = sp.album(url)
        name = info['name']
        tracks = info['tracks']['items']
        for t in tracks:
            t['album'] = info
    elif "track" in url:
        info = sp.track(url)
        name = info['name']
        tracks = [info]
    else:
        print("❌ Invalid Spotify link.")
        return

    print(f"🎧 Found {len(tracks)} tracks in '{name}'")

    folder = f"/content/{name.replace('/', '_')}"
    os.makedirs(folder, exist_ok=True)

    q = queue.Queue()
    for t in tracks:
        q.put(t)

    threads = []
    for _ in range(min(max_threads, len(tracks))):
        t = threading.Thread(target=download_worker, args=(q, folder))
        t.start()
        threads.append(t)
        sleep(0.3)

    for t in threads:
        t.join()

    zip_path = f"/content/{name.replace('/', '_')}.zip"
    shutil.make_archive(zip_path[:-4], 'zip', folder)
    print(f"\n✅ All done! ZIP: {zip_path}")
    files.download(zip_path)
    shutil.rmtree(folder, ignore_errors=True)

spotify_link = input("🎵 Paste Spotify playlist/album/track URL: ").strip()
download_spotify_from_youtube(spotify_link, max_threads=5)
